In [1]:
import sys
sys.path.append("../")

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")

In [4]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

from gradslam.datasets.tum import TUM

from datasets.llff import LLFFDataset
from datasets.llff import *

In [5]:
root_dir = "/scratch/saksham/data/tum/"

In [6]:
from datasets.colmap_utils import \
    read_cameras_binary, read_images_binary, read_points3d_binary
from datasets.tum import TUMRGBDDataset

In [7]:
# tum_path = "/scratch/saksham/data/tum/rgbd_dataset_freiburg1_desk/"
# rgb_depth_mapping = TUMRGBDDataset(tum_path).rgb_d_mapping

In [8]:
# img_wh = (504, 378)

# camdata = read_cameras_binary(os.path.join(root_dir, 'sparse/0/cameras.bin'))
# H = camdata[1].height
# W = camdata[1].width
# focal = camdata[1].params[0] * img_wh[0]/W

In [9]:
dataset = TUM(root_dir, sequences = ("../sequences.txt"), seqlen = 101, start = 56)

In [10]:
colors, depths, intrinsics, poses, transforms, names, timestamps = dataset[0]

In [11]:
num_images = colors.shape[0]

In [30]:
# import imageio
# imageio.mimsave(os.path.join("../test", 'test.gif'), imgs, fps=30)

In [12]:
timestamps

'rgb 1305031454.659528 depth 1305031454.669573 pose 1305031454.6695\nrgb 1305031454.691884 depth 1305031454.702030 pose 1305031454.6995\nrgb 1305031454.727659 depth 1305031454.740764 pose 1305031454.7394\nrgb 1305031454.759732 depth 1305031454.772865 pose 1305031454.7695\nrgb 1305031454.791641 depth 1305031454.802574 pose 1305031454.7995\nrgb 1305031454.827570 depth 1305031454.840500 pose 1305031454.8395\nrgb 1305031454.859620 depth 1305031454.870269 pose 1305031454.8694\nrgb 1305031454.891764 depth 1305031454.901065 pose 1305031454.8995\nrgb 1305031454.927567 depth 1305031454.940240 pose 1305031454.9394\nrgb 1305031454.959648 depth 1305031454.973081 pose 1305031454.9695\nrgb 1305031454.991937 depth 1305031455.010759 pose 1305031455.0098\nrgb 1305031455.027799 depth 1305031455.040446 pose 1305031455.0395\nrgb 1305031455.059636 depth 1305031455.074282 pose 1305031455.0695\nrgb 1305031455.091700 depth 1305031455.110340 pose 1305031455.1096\nrgb 1305031455.127695 depth 1305031455.142700 p

In [31]:
focal = 525
H = 480
W = 640

img_wh = (640, 480)

In [32]:
np.save("tum.npy", poses[:30])

In [17]:
poses = np.concatenate([poses[..., 0:1], -poses[..., 1:3], poses[..., 3:4]], -1)

In [18]:
poses, _ = center_poses(poses)

In [20]:
poses.shape

(30, 3, 4)

In [21]:
distances_from_center = np.linalg.norm(poses[..., 3], axis=1)

In [22]:
distances_from_center

array([0.18923801, 0.17882994, 0.17020191, 0.16133864, 0.14888205,
       0.13866221, 0.12807605, 0.1133484 , 0.10174344, 0.08953152,
       0.07296531, 0.06114183, 0.05089928, 0.04145309, 0.03819383,
       0.03999648, 0.04716561, 0.05450364, 0.06341676, 0.07298805,
       0.08622455, 0.09586377, 0.10975461, 0.12008307, 0.13056795,
       0.14478923, 0.15536617, 0.16477215, 0.17729937, 0.18645279])

In [23]:
val_idx = np.argmin(distances_from_center) # choose val image as the closest to
                                                   # center image

In [24]:
np.percentile(depths[depths>0],0.1)

0.7567999958992004

In [25]:
near_original = depths[depths>0].min()

In [26]:
scale_factor = near_original*0.75

In [27]:
scale_factor

tensor(0.4004)

In [28]:
depths = depths/scale_factor
poses = poses/scale_factor

In [29]:
directions = get_ray_directions(img_wh[1], img_wh[0], focal) # (H, W, 3)

In [30]:
img = colors[index]
depth = depths[index]

In [50]:
mask = 1 - (depth==0).long()

In [55]:
c2w = torch.FloatTensor(poses[index])

In [72]:
img = Image.fromarray(img.numpy().astype('uint8')).convert('RGB')

In [73]:
from datasets.TUM import TUMDataset

In [76]:
data = TUMDataset(root_dir, sequences = "../sequences.txt")